## 1. Setup and Imports

In [76]:
# Standard libraries
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path
import os

# Visualization libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Optional: pydeck for advanced 3D visualization
try:
    import pydeck as pdk
    PYDECK_AVAILABLE = True
except ImportError:
    PYDECK_AVAILABLE = False
    print("⚠️ pydeck not available. Install with: pip install pydeck")

print("✓ Imports successful")
print(f"  Plotly available: ✓")
print(f"  Pydeck available: {'✓' if PYDECK_AVAILABLE else '✗'}")

✓ Imports successful
  Plotly available: ✓
  Pydeck available: ✓


## 2. Configuration

Set paths to your data files.

In [77]:
# Data paths
maz_shapefile_path = r"C:\GitHub\tm2py-utils\tm2py_utils\inputs\maz_taz\shapefiles"
trip_data_path = r"E:\2023-tm22-dev-version-05\ctramp_output"

# Display options
pd.options.display.max_rows = 10

print(f"MAZ Shapefile path: {maz_shapefile_path}")
print(f"Trip data path: {trip_data_path}")

MAZ Shapefile path: C:\GitHub\tm2py-utils\tm2py_utils\inputs\maz_taz\shapefiles
Trip data path: E:\2023-tm22-dev-version-05\ctramp_output


## 3. Load MAZ Data

Load the MAZ shapefile and extract zone information.

In [78]:
# Load MAZ shapefile
shapefile_path = os.path.join(maz_shapefile_path, "mazs_TM2_2_5.shp")
print(f"Loading shapefile: {shapefile_path}")

if os.path.exists(shapefile_path):
    mazs_gdf = gpd.read_file(shapefile_path)
    print(f"✓ Loaded {len(mazs_gdf):,} MAZ zones")
    print(f"  Columns: {list(mazs_gdf.columns)}")
    print(f"  CRS: {mazs_gdf.crs}")
    mazs_gdf.head(3)
else:
    print(f"❌ Shapefile not found at: {shapefile_path}")

Loading shapefile: C:\GitHub\tm2py-utils\tm2py_utils\inputs\maz_taz\shapefiles\mazs_TM2_2_5.shp
✓ Loaded 39,586 MAZ zones
  Columns: ['MAZ_NODE', 'COUNTYFP10', 'ALAND10', 'AWATER10', 'blockcount', 'TAZ_NODE', 'partcount', 'PERIM_MI', 'AREA_SQMI', 'psq_overa', 'acres', 'MAZ_X', 'MAZ_Y', 'MAZ_SEQ', 'geometry']
  CRS: EPSG:4326


In [79]:
# Identify MAZ ID column (adjust based on your shapefile)
print("Available columns:", mazs_gdf.columns.tolist())
print("\nColumn data types:")
for col in mazs_gdf.columns:
    if col != 'geometry':
        print(f"  {col}: {mazs_gdf[col].dtype} | Sample values: {mazs_gdf[col].head(3).tolist()}")

# Try to identify the MAZ ID column automatically
potential_id_cols = [col for col in mazs_gdf.columns if 'maz' in col.lower() or 'id' in col.lower() or 'seq' in col.lower()]
print(f"\nPotential MAZ ID columns: {potential_id_cols}")

# Use the first numeric column that looks like an ID, or default to first column
if potential_id_cols:
    maz_id_column = potential_id_cols[0]
else:
    # Fall back to first non-geometry column
    maz_id_column = [col for col in mazs_gdf.columns if col != 'geometry'][0]

print(f"\n✓ Using '{maz_id_column}' as MAZ ID")
print(f"  Range: {mazs_gdf[maz_id_column].min()} to {mazs_gdf[maz_id_column].max()}")
print(f"  Count: {len(mazs_gdf[maz_id_column].unique())} unique values")

Available columns: ['MAZ_NODE', 'COUNTYFP10', 'ALAND10', 'AWATER10', 'blockcount', 'TAZ_NODE', 'partcount', 'PERIM_MI', 'AREA_SQMI', 'psq_overa', 'acres', 'MAZ_X', 'MAZ_Y', 'MAZ_SEQ', 'geometry']

Column data types:
  MAZ_NODE: int64 | Sample values: [10001, 10002, 10003]
  COUNTYFP10: object | Sample values: ['075', '075', '075']
  ALAND10: int64 | Sample values: [16958, 16823, 17020]
  AWATER10: int64 | Sample values: [0, 0, 0]
  blockcount: int64 | Sample values: [1, 1, 1]
  TAZ_NODE: int64 | Sample values: [56, 56, 10]
  partcount: int32 | Sample values: [1, 1, 1]
  PERIM_MI: float64 | Sample values: [0.339828227125872, 0.338450933593434, 0.339226492786627]
  AREA_SQMI: float64 | Sample values: [0.006546485663391, 0.00649437763712, 0.006570492707147]
  psq_overa: float64 | Sample values: [17.6404913856783, 17.638185034934782, 17.513848433793825]
  acres: float64 | Sample values: [4.190409354413051, 4.157050157405989, 4.205729874519998]
  MAZ_X: float64 | Sample values: [-122.441076

## 4. Load Trip Data

In [80]:
# Load trip data
trips = pd.read_csv(os.path.join(trip_data_path, "indivTripData_1.csv"))
print(f"✓ Loaded {len(trips):,} trips")
print(f"  Columns: {list(trips.columns)}")
trips.head(3)

✓ Loaded 240,622 trips
  Columns: ['hh_id', 'person_id', 'person_num', 'tour_id', 'stop_id', 'inbound', 'tour_purpose', 'orig_purpose', 'dest_purpose', 'orig_mgra', 'dest_mgra', 'trip_dist', 'parking_mgra', 'stop_period', 'trip_mode', 'tour_mode', 'tranpath_rnum', 'sampleRate', 'avAvailable']


,hh_id,person_id,person_num,tour_id,stop_id,inbound,tour_purpose,orig_purpose,dest_purpose,orig_mgra,dest_mgra,trip_dist,parking_mgra,stop_period,trip_mode,tour_mode,tranpath_rnum,sampleRate,avAvailable
0,1562223,3806279,1,0,-1,0,Work,Home,Work,3,3349,1.923149,0,7,10,10,-1.0,0.01,0
1,1562223,3806279,1,0,-1,1,Work,Work,Home,3349,3,1.936884,0,40,10,10,-1.0,0.01,0
2,1562223,3806279,1,1,-1,0,Work,Home,Work,3,3349,1.923149,0,1,10,10,-1.0,0.01,0


In [81]:
# Inspect trip data columns to verify field names
print("All columns in trips data:")
print(trips.columns.tolist())
print(f"\nChecking for key columns:")
print(f"  'sample_rate' exists: {'sample_rate' in trips.columns}")
print(f"  'sampleRate' exists: {'sampleRate' in trips.columns}")
print(f"  'dest_purpose' exists: {'dest_purpose' in trips.columns}")
print(f"  'stop_period' exists: {'stop_period' in trips.columns}")
print(f"  'hour' exists: {'hour' in trips.columns}")

All columns in trips data:
['hh_id', 'person_id', 'person_num', 'tour_id', 'stop_id', 'inbound', 'tour_purpose', 'orig_purpose', 'dest_purpose', 'orig_mgra', 'dest_mgra', 'trip_dist', 'parking_mgra', 'stop_period', 'trip_mode', 'tour_mode', 'tranpath_rnum', 'sampleRate', 'avAvailable']

Checking for key columns:
  'sample_rate' exists: False
  'sampleRate' exists: True
  'dest_purpose' exists: True
  'stop_period' exists: True
  'hour' exists: False


## 5. Data Filtering

Configure dataset size - use all MAZs or subset for faster testing.

In [82]:
# ============================================
# DATA SCALING OPTIONS
# ============================================
# Set NUM_MAZS to limit data for faster testing, or None for all MAZs
NUM_MAZS = None  # Use None for all MAZs, or set to a number like 100 for testing

if NUM_MAZS is not None:
    mazs_gdf = mazs_gdf.head(NUM_MAZS).copy()
    print(f"⚠️  Using subset: {len(mazs_gdf)} MAZ zones")
else:
    print(f"✓ Using all {len(mazs_gdf):,} MAZ zones")

# Filter trips to only include those with destinations in our MAZ set
valid_maz_ids = set(mazs_gdf[maz_id_column])
trips_valid = trips[trips['dest_mgra'].isin(valid_maz_ids)].copy()

# Apply sample rate expansion to get actual trip counts
# trip_weight represents the actual number of trips this record represents
trips_valid['trip_weight'] = 1 / trips_valid['sampleRate']

print(f"📊 DATA SUMMARY:")
print(f"   MAZ zones: {len(mazs_gdf):,}")
print(f"   Trip records: {len(trips_valid):,}")
print(f"   Expanded trips: {trips_valid['trip_weight'].sum():,.0f}")
if NUM_MAZS is not None:
    print(f"   💡 Tip: Set NUM_MAZS = None for full dataset")

✓ Using all 39,586 MAZ zones
📊 DATA SUMMARY:
   MAZ zones: 39,586
   Trip records: 28,006
   Expanded trips: 2,800,600


In [83]:
# Diagnostic: Check MAZ ID matching
print("MAZ ID Diagnostics:")
print(f"  Shapefile MAZ IDs - Type: {mazs_gdf[maz_id_column].dtype}")
print(f"  Shapefile MAZ IDs - Sample: {mazs_gdf[maz_id_column].head().tolist()}")
print(f"  Shapefile MAZ IDs - Range: {mazs_gdf[maz_id_column].min()} to {mazs_gdf[maz_id_column].max()}")
print(f"\n  Trip dest_mgra - Type: {trips['dest_mgra'].dtype}")
print(f"  Trip dest_mgra - Sample: {trips['dest_mgra'].head().tolist()}")
print(f"  Trip dest_mgra - Range: {trips['dest_mgra'].min()} to {trips['dest_mgra'].max()}")
print(f"\n  Unique MAZ IDs in shapefile: {len(mazs_gdf[maz_id_column].unique()):,}")
print(f"  Unique dest_mgra in trips: {len(trips['dest_mgra'].unique()):,}")
print(f"  MAZ IDs that match: {len(set(mazs_gdf[maz_id_column]) & set(trips['dest_mgra'])):,}")

MAZ ID Diagnostics:
  Shapefile MAZ IDs - Type: int64
  Shapefile MAZ IDs - Sample: [10001, 10002, 10003, 10004, 10005]
  Shapefile MAZ IDs - Range: 10001 to 814506

  Trip dest_mgra - Type: int64
  Trip dest_mgra - Sample: [3349, 3, 3349, 3, 2185]
  Trip dest_mgra - Range: 1 to 39726

  Unique MAZ IDs in shapefile: 39,586
  Unique dest_mgra in trips: 27,681
  MAZ IDs that match: 2,955


## 6. Prepare Geographic Data

Extract centroids and reproject to Web Mercator for visualization.

In [84]:
# Calculate centroids in original CRS
mazs_gdf['centroid'] = mazs_gdf.geometry.centroid

# Reproject to Web Mercator (EPSG:3857) for web mapping
mazs_gdf = mazs_gdf.to_crs(epsg=3857)

# Also get lat/lon for some visualizations (EPSG:4326)
mazs_wgs84 = mazs_gdf.to_crs(epsg=4326)
mazs_wgs84['lon'] = mazs_wgs84.centroid.x
mazs_wgs84['lat'] = mazs_wgs84.centroid.y

# Extract coordinates
maz_ids = mazs_gdf[maz_id_column].values
centroids = mazs_gdf['centroid']
maz_points = np.array([[pt.x, pt.y, 0.0] for pt in centroids])

print(f"Coordinate ranges (Web Mercator):")
print(f"  X: {maz_points[:,0].min():.0f} to {maz_points[:,0].max():.0f}")
print(f"  Y: {maz_points[:,1].min():.0f} to {maz_points[:,1].max():.0f}")

C:\Users\mtcpb\AppData\Local\Temp\2\ipykernel_8584\1571466424.py:2: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\mtcpb\AppData\Local\Temp\2\ipykernel_8584\1571466424.py:9: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\mtcpb\AppData\Local\Temp\2\ipykernel_8584\1571466424.py:10: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




Coordinate ranges (Web Mercator):
  X: -124 to -121
  Y: 37 to 39


## 7. Process Trip Data

Add time windows and trip purposes.

In [85]:
# Process trip purposes
print("Unique trip purposes:", trips_valid['dest_purpose'].unique())

# Add hour calculation from stop_period (assuming 30-minute intervals starting at 3 AM)
trips_valid['start_time'] = (trips_valid['stop_period'] - 1) * 30 + 180  # minutes from midnight
trips_valid['hour'] = trips_valid['start_time'] // 60

# Map purposes to integers for coloring
unique_purposes = trips_valid['dest_purpose'].dropna().unique()
purposes = {x: i for i, x in enumerate(sorted(unique_purposes))}
print(f"Trip purposes: {list(purposes.keys())}")

trips_valid.head()

trips_valid['p_type'] = trips_valid['dest_purpose'].map(purposes).fillna(-1).astype(int)

Unique trip purposes: ['Maintenance' 'Discretionary' 'Escort' 'Shop' 'work related' 'Work'
 'Visiting' 'Work-Based' 'Eating Out' 'School' 'University' 'Home']
Trip purposes: ['Discretionary', 'Eating Out', 'Escort', 'Home', 'Maintenance', 'School', 'Shop', 'University', 'Visiting', 'Work', 'Work-Based', 'work related']


## 8. Join Trip Locations to MAZ Coordinates

In [86]:
# Create lookup for MAZ coordinates
maz_lookup = mazs_wgs84[[maz_id_column, 'lon', 'lat']].copy()
maz_lookup.columns = ['maz_id', 'lon', 'lat']

# Diagnostic: Check data before join
print("=" * 60)
print("JOIN DIAGNOSTICS - Before Merge")
print("=" * 60)
print(f"Trip data:")
print(f"  Total trips: {len(trips_valid):,}")
print(f"  Unique dest_mgra: {trips_valid['dest_mgra'].nunique():,}")
print(f"  dest_mgra dtype: {trips_valid['dest_mgra'].dtype}")
print(f"  dest_mgra range: {trips_valid['dest_mgra'].min()} to {trips_valid['dest_mgra'].max()}")
print(f"  Sample dest_mgra: {trips_valid['dest_mgra'].head(10).tolist()}")

print(f"\nMAZ lookup data:")
print(f"  Total MAZ zones: {len(maz_lookup):,}")
print(f"  Unique maz_id: {maz_lookup['maz_id'].nunique():,}")
print(f"  maz_id dtype: {maz_lookup['maz_id'].dtype}")
print(f"  maz_id range: {maz_lookup['maz_id'].min()} to {maz_lookup['maz_id'].max()}")
print(f"  Sample maz_id: {maz_lookup['maz_id'].head(10).tolist()}")

print(f"\nOverlap check:")
trip_ids = set(trips_valid['dest_mgra'].unique())
maz_ids = set(maz_lookup['maz_id'].unique())
matching_ids = trip_ids & maz_ids
print(f"  Trip dest_mgra values: {len(trip_ids):,}")
print(f"  MAZ maz_id values: {len(maz_ids):,}")
print(f"  Matching IDs: {len(matching_ids):,}")
print(f"  Non-matching trip IDs: {len(trip_ids - maz_ids):,}")
if len(trip_ids - maz_ids) > 0:
    print(f"  Sample non-matching: {list(trip_ids - maz_ids)[:10]}")
print("=" * 60)

# Join destination coordinates
trips_with_coords = trips_valid.merge(
    maz_lookup,
    left_on='dest_mgra',
    right_on='maz_id',
    how='left'
)

# Check results after merge
print("\nJOIN RESULTS:")
print(f"  Trips after merge: {len(trips_with_coords):,}")
print(f"  Trips with non-null lon: {trips_with_coords['lon'].notna().sum():,}")
print(f"  Trips with non-null lat: {trips_with_coords['lat'].notna().sum():,}")
print(f"  Trips with both coordinates: {((trips_with_coords['lon'].notna()) & (trips_with_coords['lat'].notna())).sum():,}")

# Show sample of trips without coordinates
missing_coords = trips_with_coords[trips_with_coords['lon'].isna() | trips_with_coords['lat'].isna()]
if len(missing_coords) > 0:
    print(f"\n⚠️  WARNING: {len(missing_coords):,} trips missing coordinates")
    print(f"  Sample missing dest_mgra values: {missing_coords['dest_mgra'].head(10).tolist()}")

# Drop trips without valid coordinates
trips_with_coords = trips_with_coords.dropna(subset=['lon', 'lat'])

print(f"\n✓ Final trips with valid coordinates: {len(trips_with_coords):,}")
print("=" * 60)
trips_with_coords.head()

JOIN DIAGNOSTICS - Before Merge
Trip data:
  Total trips: 28,006
  Unique dest_mgra: 2,955
  dest_mgra dtype: int64
  dest_mgra range: 10001 to 17386
  Sample dest_mgra: [17375, 12574, 12242, 10993, 17378, 13930, 16801, 16021, 10277, 15160]

MAZ lookup data:
  Total MAZ zones: 39,586
  Unique maz_id: 39,586
  maz_id dtype: int64
  maz_id range: 10001 to 814506
  Sample maz_id: [10001, 10002, 10003, 10004, 10005, 10006, 10007, 10009, 10012, 10017]

Overlap check:
  Trip dest_mgra values: 2,955
  MAZ maz_id values: 39,586
  Matching IDs: 2,955
  Non-matching trip IDs: 0

JOIN RESULTS:
  Trips after merge: 28,006
  Trips with non-null lon: 28,006
  Trips with non-null lat: 28,006
  Trips with both coordinates: 28,006

✓ Final trips with valid coordinates: 28,006


,hh_id,person_id,person_num,tour_id,stop_id,inbound,tour_purpose,orig_purpose,dest_purpose,orig_mgra,...,tranpath_rnum,sampleRate,avAvailable,trip_weight,start_time,hour,p_type,maz_id,lon,lat
0,1603540,3891781,3,0,-1,0,Maintenance,Home,Maintenance,181,...,-1.0,0.01,0,100.0,630,10,4,17375,-122.468533,37.757396
1,1384796,3496578,1,1,-1,0,Discretionary,Home,Discretionary,327,...,-1.0,0.01,0,100.0,660,11,0,12574,-122.419759,37.781591
2,1669571,4039189,3,0,0,1,Work,Work,Escort,6438,...,-1.0,0.01,0,100.0,900,15,2,12242,-122.460873,37.717833
3,1408647,3545869,1,0,-1,0,Discretionary,Home,Discretionary,1134,...,-1.0,0.01,0,100.0,450,7,0,10993,-122.430182,37.791675
4,1654285,4001006,4,0,-1,0,Discretionary,Home,Discretionary,1241,...,-1.0,0.01,0,100.0,810,13,0,17378,-122.459451,37.765745


## 9. Visualization 1: Trip Destinations by Purpose (Plotly Scattermapbox)

In [87]:
# Aggregate trips by destination and purpose for sizing (using trip weights)
trips_agg = trips_with_coords.groupby(['dest_mgra', 'dest_purpose', 'lon', 'lat'])['trip_weight'].sum().reset_index(name='trip_count')

fig = px.scatter_mapbox(
    trips_agg,
    lat='lat',
    lon='lon',
    color='dest_purpose',
    size='trip_count',
    hover_data=['dest_mgra', 'dest_purpose', 'trip_count'],
    zoom=10,
    height=600,
    title=f'Trip Destinations by Purpose - 2023 Travel Model 2 outputs (draft)<br><sub>{trips_with_coords["trip_weight"].sum():,.0f} trips</sub>'
)

fig.update_layout(
    mapbox_style="carto-positron",  # Clean, minimal style
    margin={"r": 0, "t": 60, "l": 0, "b": 0}
)

# Save static version
fig.write_html('trip_destinations_by_purpose.html')
fig.show()

print("✓ Saved: trip_destinations_by_purpose.html")

C:\Users\mtcpb\AppData\Local\Temp\2\ipykernel_8584\344635974.py:4: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



✓ Saved: trip_destinations_by_purpose.html


## 10. Visualization 2: Trips by Time of Day

In [88]:
# Count trips by hour and purpose (using trip weights)
hourly_trips = trips_with_coords.groupby(['hour', 'dest_purpose'])['trip_weight'].sum().reset_index(name='count')

fig = px.line(
    hourly_trips,
    x='hour',
    y='count',
    color='dest_purpose',
    title='Trips by Hour and Purpose - 2023 Travel Model 2 outputs (draft)',
    labels={'hour': 'Hour of Day', 'count': 'Number of Trips'}
)

fig.update_layout(height=400)

# Save static version
fig.write_html('trips_by_hour.html')
fig.write_image('trips_by_hour.png', width=1200, height=500)
fig.show()

print("✓ Saved: trips_by_hour.html and trips_by_hour.png")

ValueError: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido


## 11. Visualization 3: MAZ Zones Heatmap

In [ ]:
# Count trips by destination MAZ (using trip weights)
maz_trip_counts = trips_with_coords.groupby('dest_mgra')['trip_weight'].sum().reset_index(name='trip_count')

# Join to MAZ coordinates
maz_with_counts = mazs_wgs84.merge(
    maz_trip_counts,
    left_on=maz_id_column,
    right_on='dest_mgra',
    how='left'
)
maz_with_counts['trip_count'] = maz_with_counts['trip_count'].fillna(0)

fig = px.scatter_mapbox(
    maz_with_counts,
    lat='lat',
    lon='lon',
    color='trip_count',
    size='trip_count',
    color_continuous_scale='YlOrRd',
    hover_data=[maz_id_column, 'trip_count'],
    zoom=10,
    height=600,
    title='Trip Destinations Heatmap by MAZ - 2023 Travel Model 2 outputs (draft)'
)

fig.update_layout(
    mapbox_style="carto-positron",  # Clean, minimal style
    margin={"r": 0, "t": 60, "l": 0, "b": 0}
)

# Save static version
fig.write_html('trip_heatmap.html')
fig.show()

print("✓ Saved: trip_heatmap.html")

C:\Users\mtcpb\AppData\Local\Temp\2\ipykernel_8584\3860769832.py:13: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



## 12b. 3D Map with Trip Columns

3D map visualization with columns where height = trip count and color = trip purpose.

In [ ]:
# Prepare data: aggregate trips by destination and purpose (using trip weights)
trip_summary = trips_with_coords.groupby(['dest_mgra', 'dest_purpose'])['trip_weight'].sum().reset_index(name='trip_count')

# Join with coordinates
trip_summary = trip_summary.merge(
    mazs_wgs84[[maz_id_column, 'lon', 'lat']],
    left_on='dest_mgra',
    right_on=maz_id_column,
    how='left'
)

# Map purposes to colors (RGB values)
purpose_colors = {
    'Home': [255, 0, 0],      # Red
    'Work': [0, 0, 255],       # Blue
    'University': [255, 165, 0], # Orange
    'School': [255, 255, 0],   # Yellow
    'Escort': [128, 0, 128],   # Purple
    'Shop': [0, 255, 0],       # Green
    'Maintenance': [0, 255, 255], # Cyan
    'EatingOut': [255, 192, 203], # Pink
    'Visiting': [165, 42, 42], # Brown
    'Discretionary': [128, 128, 128], # Gray
    'WorkBased': [0, 128, 128] # Teal
}

# Add RGB color column
trip_summary['color'] = trip_summary['dest_purpose'].map(
    lambda x: purpose_colors.get(x, [200, 200, 200])
)

# Scale trip counts for better visualization (height in meters)
trip_summary['elevation'] = trip_summary['trip_count'] * 50  # Adjust multiplier as needed

print(f"Prepared {len(trip_summary):,} destination-purpose combinations")
print(f"Purposes: {trip_summary['dest_purpose'].unique()}")
trip_summary.head()

Prepared 7,617 destination-purpose combinations
Purposes: ['Discretionary' 'Maintenance' 'School' 'Visiting' 'Escort' 'Shop' 'Work'
 'Home' 'University' 'Eating Out' 'Work-Based' 'work related']


,dest_mgra,dest_purpose,trip_count,MAZ_NODE,lon,lat,color,elevation
0,10001,Discretionary,300.0,10001,-122.441076,37.750067,"[128, 128, 128]",15000.0
1,10001,Maintenance,300.0,10001,-122.441076,37.750067,"[0, 255, 255]",15000.0
2,10001,School,100.0,10001,-122.441076,37.750067,"[255, 255, 0]",5000.0
3,10001,Visiting,100.0,10001,-122.441076,37.750067,"[165, 42, 42]",5000.0
4,10002,Discretionary,100.0,10002,-122.438934,37.750197,"[128, 128, 128]",5000.0


In [ ]:
# Create 3D map with Pydeck - Columns on Map
if PYDECK_AVAILABLE:
    # Choose your map style (uncomment the one you want):
    # map_style = 'mapbox://styles/mapbox/dark-v10'          # Dark background (default)
    # map_style = 'mapbox://styles/mapbox/light-v10'         # Light background
    # map_style = 'mapbox://styles/mapbox/streets-v11'       # Street map
    # map_style = 'mapbox://styles/mapbox/outdoors-v11'      # Outdoors/terrain
    # map_style = 'mapbox://styles/mapbox/satellite-v9'      # Satellite imagery
    # map_style = 'mapbox://styles/mapbox/satellite-streets-v11'  # Satellite with streets
    # map_style = 'mapbox://styles/mapbox/navigation-day-v1' # Navigation style
    map_style = 'mapbox://styles/mapbox/light-v10'  # Currently selected
    
    # Create column layer with trip data
    column_layer = pdk.Layer(
        'ColumnLayer',
        data=trip_summary,
        get_position='[lon, lat]',
        get_elevation='elevation',
        elevation_scale=1,
        radius=100,  # Column radius in meters
        get_fill_color='color',
        pickable=True,
        auto_highlight=True,
    )
    
    # Set initial view state
    view_state = pdk.ViewState(
        longitude=trip_summary['lon'].mean(),
        latitude=trip_summary['lat'].mean(),
        zoom=11,
        pitch=45,
        bearing=0
    )
    
    # Create deck with tooltip
    r = pdk.Deck(
        layers=[column_layer],
        initial_view_state=view_state,
        tooltip={
            "html": "<b>MAZ:</b> {dest_mgra}<br/>"
                    "<b>Purpose:</b> {dest_purpose}<br/>"
                    "<b>Trips:</b> {trip_count}",
            "style": {"backgroundColor": "steelblue", "color": "white"}
        },
        map_style=map_style
    )
    
    # Save to HTML for viewing
    r.to_html('trip_3d_columns_map.html')
    print("✓ 3D column map saved to: trip_3d_columns_map.html")
    print("✓ 2023 Travel Model 2 outputs (draft)")
    print("✓ Column height = trip count, Column color = trip purpose")
    
    r
else:
    print("❌ Pydeck is required for 3D map visualization")
    print("   Install with: pip install pydeck")
    print("\n   Alternative: Using Plotly scattermapbox with 3D-like effects...")
    
    # Fallback to Plotly mapbox
    fig = px.scatter_mapbox(
        trip_summary,
        lat='lat',
        lon='lon',
        color='dest_purpose',
        size='trip_count',
        hover_data=['dest_mgra', 'dest_purpose', 'trip_count'],
        zoom=11,
        height=700,
        title='Trip Destinations by Purpose (2D Map View)'
    )
    
    fig.update_layout(
        mapbox_style="carto-darkmatter",
        margin={"r": 0, "t": 40, "l": 0, "b": 0}
    )
    
    fig.show()

✓ 3D column map saved to: trip_3d_columns_map.html
✓ Column height = trip count, Column color = trip purpose


In [ ]:
# Alternative: Create legend for trip purposes
if PYDECK_AVAILABLE:
    print("\n📊 LEGEND - Trip Purpose Colors:")
    print("=" * 40)
    for purpose, color in purpose_colors.items():
        if purpose in trip_summary['dest_purpose'].values:
            count = trip_summary[trip_summary['dest_purpose'] == purpose]['trip_count'].sum()
            print(f"  {purpose:20s} RGB{str(tuple(color)):20s} ({count:,} trips)")
    print("=" * 40)
    print(f"\nℹ️  Rotate the map by holding Ctrl + dragging")
    print(f"ℹ️  Change pitch by holding Shift + dragging")
    print(f"ℹ️  Open trip_3d_columns_map.html in browser for interactive view")


📊 LEGEND - Trip Purpose Colors:
  Home                 RGB(255, 0, 0)          (859,500.0 trips)
  Work                 RGB(0, 0, 255)          (262,800.0 trips)
  University           RGB(255, 165, 0)        (37,200.0 trips)
  School               RGB(255, 255, 0)        (170,000.0 trips)
  Escort               RGB(128, 0, 128)        (332,400.0 trips)
  Shop                 RGB(0, 255, 0)          (283,100.0 trips)
  Maintenance          RGB(0, 255, 255)        (327,400.0 trips)
  Visiting             RGB(165, 42, 42)        (133,000.0 trips)
  Discretionary        RGB(128, 128, 128)      (198,700.0 trips)

ℹ️  Rotate the map by holding Ctrl + dragging
ℹ️  Change pitch by holding Shift + dragging
ℹ️  Open trip_3d_columns_map.html in browser for interactive view


## 14. Summary Statistics

In [ ]:
print("=" * 60)
print("TRIP VISUALIZATION SUMMARY")
print("=" * 60)
print(f"Total MAZ Zones: {len(mazs_gdf):,}")
print(f"Total Trip Records: {len(trips_with_coords):,}")
print(f"Total Expanded Trips: {trips_with_coords['trip_weight'].sum():,.0f}")
print(f"\nTrips by Purpose (expanded):")
print(trips_with_coords.groupby('dest_purpose')['trip_weight'].sum().sort_values(ascending=False))
print(f"\nPeak Hours:")
print(trips_with_coords['hour'].value_counts().head(5))
print("=" * 60)

TRIP VISUALIZATION SUMMARY
Total MAZ Zones: 39,586
Total Trip Records: 28,006
Total Expanded Trips: 2,800,600

Trips by Purpose (expanded):
dest_purpose
Home            859500.0
Escort          332400.0
Maintenance     327400.0
Shop            283100.0
Work            262800.0
                  ...   
Eating Out      137700.0
Visiting        133000.0
Work-Based       40000.0
University       37200.0
work related     18800.0
Name: trip_weight, Length: 12, dtype: float64

Peak Hours:
hour
6     3213
13    2589
15    2246
14    2044
16    1970
Name: count, dtype: int64
